In [1]:
#pip install git+https://github.com/NCAR/xcollection.git

In [1]:
import xcollection as xc

import concurrent.futures
import typing

import warnings
from copy import deepcopy

import dask
import pandas as pd
import pydantic
import xarray as xr
import xcollection as xc
from fastprogress.fastprogress import progress_bar
from intake.catalog import Catalog

from intake_esm.cat import ESMCatalogModel
from intake_esm.derived import DerivedVariableRegistry, default_registry
from intake_esm.source import ESMDataSource

import ast
import intake
import pytest

/Users/jkent/miniconda3/envs/xcollection-dev/lib/python3.9/site-packages/fastprogress/fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
import intake_esm
print(intake_esm.__version__)

2021.8.17


In [3]:
#col = intake.open_esm_datastore("glade-cmip6.json") # No such file?
# cat = col.search(
#     source_id=["BCC-CSM2-MR", "CNRM-CM6-1", "CNRM-ESM2-1"],
#     experiment_id=["historical", "ssp585"],
#     variable_id="pr",
#     table_id="Amon",
#     grid_label="gn",
#     )
# dsets = cat.to_dataset_dict()

In [4]:
#conda list intake

In [5]:
#pip show intake_esm

In [6]:
#import sys
#print(sys.executable)

In [7]:
#print(intake.registry)

In [8]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
print(type(col))
col

<class 'intake_esm.core.esm_datastore'>


,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,709
grid_label,10
zstore,521075
dcpp_init_year,60


In [9]:
cat = col.search(
    experiment_id=["historical", "ssp585"],
    table_id="Oyr",
    variable_id="o2",
    grid_label="gn",
)

cat

,unique
activity_id,2
institution_id,13
source_id,15
experiment_id,2
member_id,47
table_id,1
variable_id,1
grid_label,1
zstore,173
dcpp_init_year,0


In [10]:
cat.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,IPSL,IPSL-CM6A-LR,historical,r24i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
1,CMIP,IPSL,IPSL-CM6A-LR,historical,r25i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
2,CMIP,IPSL,IPSL-CM6A-LR,historical,r10i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
3,CMIP,IPSL,IPSL-CM6A-LR,historical,r11i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
4,CMIP,IPSL,IPSL-CM6A-LR,historical,r21i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803


In [11]:
dset_dict = cat.to_dataset_dict(
    zarr_kwargs={"consolidated": True, "decode_times": True}
)

/var/folders/dd/_xm_pbpd3flgbvbnt7qhd70snnbpj_/T/ipykernel_22903/1715370549.py:1: DeprecationWarning: cdf_kwargs and zarr_kwargs are deprecated and will be removed in a future version. Please use xarray_open_kwargs instead.
  dset_dict = cat.to_dataset_dict(



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [12]:
dset_dict.keys()

dict_keys(['CMIP.IPSL.IPSL-CM5A2-INCA.historical.Oyr.gn', 'ScenarioMIP.CMCC.CMCC-ESM2.ssp585.Oyr.gn', 'CMIP.EC-Earth-Consortium.EC-Earth3-CC.historical.Oyr.gn', 'ScenarioMIP.MRI.MRI-ESM2-0.ssp585.Oyr.gn', 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp585.Oyr.gn', 'CMIP.CMCC.CMCC-ESM2.historical.Oyr.gn', 'ScenarioMIP.DWD.MPI-ESM1-2-HR.ssp585.Oyr.gn', 'ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.Oyr.gn', 'CMIP.NCC.NorESM2-LM.historical.Oyr.gn', 'CMIP.NCC.NorESM2-MM.historical.Oyr.gn', 'CMIP.MRI.MRI-ESM2-0.historical.Oyr.gn', 'ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp585.Oyr.gn', 'CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.Oyr.gn', 'CMIP.MPI-M.MPI-ESM1-2-LR.historical.Oyr.gn', 'ScenarioMIP.DKRZ.MPI-ESM1-2-HR.ssp585.Oyr.gn', 'CMIP.CCCma.CanESM5-CanOE.historical.Oyr.gn', 'ScenarioMIP.MIROC.MIROC-ES2L.ssp585.Oyr.gn', 'ScenarioMIP.NCC.NorESM2-MM.ssp585.Oyr.gn', 'ScenarioMIP.NCC.NorESM2-LM.ssp585.Oyr.gn', 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.Oyr.gn', 'CMIP.MPI-M.MPI-ESM1-2-HR.historical.Oyr.gn

In [15]:
dset_collection = xc.Collection(dset_dict)
dset_collection

<IPython.core.display.HTML object>

In [17]:
dset_collection.keys()

dict_keys(['CMIP.IPSL.IPSL-CM5A2-INCA.historical.Oyr.gn', 'ScenarioMIP.CMCC.CMCC-ESM2.ssp585.Oyr.gn', 'CMIP.EC-Earth-Consortium.EC-Earth3-CC.historical.Oyr.gn', 'ScenarioMIP.MRI.MRI-ESM2-0.ssp585.Oyr.gn', 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp585.Oyr.gn', 'CMIP.CMCC.CMCC-ESM2.historical.Oyr.gn', 'ScenarioMIP.DWD.MPI-ESM1-2-HR.ssp585.Oyr.gn', 'ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.Oyr.gn', 'CMIP.NCC.NorESM2-LM.historical.Oyr.gn', 'CMIP.NCC.NorESM2-MM.historical.Oyr.gn', 'CMIP.MRI.MRI-ESM2-0.historical.Oyr.gn', 'ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp585.Oyr.gn', 'CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.Oyr.gn', 'CMIP.MPI-M.MPI-ESM1-2-LR.historical.Oyr.gn', 'ScenarioMIP.DKRZ.MPI-ESM1-2-HR.ssp585.Oyr.gn', 'CMIP.CCCma.CanESM5-CanOE.historical.Oyr.gn', 'ScenarioMIP.MIROC.MIROC-ES2L.ssp585.Oyr.gn', 'ScenarioMIP.NCC.NorESM2-MM.ssp585.Oyr.gn', 'ScenarioMIP.NCC.NorESM2-LM.ssp585.Oyr.gn', 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.Oyr.gn', 'CMIP.MPI-M.MPI-ESM1-2-HR.historical.Oyr.gn

In [18]:
dset_collection['CMIP.IPSL.IPSL-CM5A2-INCA.historical.Oyr.gn']

<xarray.Dataset>
Dimensions:         (y: 149, x: 182, nvertex: 4, deptht: 31, axis_nbounds: 2, member_id: 1, dcpp_init_year: 1, time: 165)
Coordinates:
    bounds_nav_lat  (y, x, nvertex) float32 dask.array<chunksize=(149, 182, 4), meta=np.ndarray>
    bounds_nav_lon  (y, x, nvertex) float32 dask.array<chunksize=(149, 182, 4), meta=np.ndarray>
  * deptht          (deptht) float32 5.0 15.0 25.0 ... 4.75e+03 5.25e+03
    deptht_bounds   (deptht, axis_nbounds) float32 dask.array<chunksize=(31, 2), meta=np.ndarray>
    nav_lat         (y, x) float32 dask.array<chunksize=(149, 182), meta=np.ndarray>
    nav_lon         (y, x) float32 dask.array<chunksize=(149, 182), meta=np.ndarray>
  * time            (time) object 1850-07-02 12:00:00 ... 2014-07-02 12:00:00
    time_bounds     (time, axis_nbounds) object dask.array<chunksize=(165, 2), meta=np.ndarray>
  * member_id       (member_id) <U8 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
Dimensions without coordinates: y, x, nvertex, axis_nbounds
Data variables:
    area            (y, x) float32 dask.array<chunksize=(149, 182), meta=np.ndarray>
    o2              (member_id, dcpp_init_year, time, deptht, y, x) float32 dask.array<chunksize=(1, 1, 34, 31, 149, 182), meta=np.ndarray>
Attributes: (12/66)
    CMIP6_CV_version:                 cv=6.2.15.1
    Conventions:                      CF-1.7 CMIP-6.2
    EXPID:                            historical
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    ...                               ...
    intake_esm_attrs/variable_id:     o2
    intake_esm_attrs/grid_label:      gn
    intake_esm_attrs/zstore:          gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM5A2-I...
    intake_esm_attrs/dcpp_init_year:  nan
    intake_esm_attrs/version:         20200729
    intake_esm_dataset_key:           CMIP.IPSL.IPSL-CM5A2-INCA.historical.Oy...

In [20]:
dset_collection.dims

AttributeError: 'Collection' object has no attribute 'dims'

In [ ]:
cat = intake.open_esm_datastore(path)